# Part 3: Prompt Engineering Basics

## Introduction

In this part, you'll experiment with different prompting techniques to improve the quality of responses from Large Language Models (LLMs). You'll compare zero-shot, one-shot, and few-shot prompting approaches and document which works best for different types of questions.

## Learning Objectives

- Understand different prompting techniques
- Compare zero-shot, one-shot, and few-shot prompting
- Analyze the impact of prompt design on response quality

## Setup

In [2]:
# Import necessary libraries
import requests
import json

## 1. Understanding Prompting Techniques

LLMs can be prompted in different ways to get better responses:

1. **Zero-shot prompting**: Asking the model a question directly without examples
2. **One-shot prompting**: Providing one example before asking your question
3. **Few-shot prompting**: Providing multiple examples before asking your question

## 2. Creating Prompting Templates

Your first task is to create templates for different prompting strategies.

In [3]:
# Define a question to experiment with
question = "What foods should be avoided by patients with gout?"

# Example for one-shot and few-shot prompting
example_q = "What are the symptoms of gout?"
example_a = "Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe."

# Examples for few-shot prompting
examples = [
    ("What are the symptoms of gout?",
     "Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe."),
    ("How is gout diagnosed?",
     "Gout is diagnosed through physical examination, medical history, blood tests for uric acid levels, and joint fluid analysis to look for urate crystals.")
]

# TODO: Create prompting templates
# Zero-shot template (just the question)
zero_shot_template = "Question: {question}\nAnswer:"

# One-shot template (one example + the question)
one_shot_template = """Question: {example_q}
Answer: {example_a}

Question: {question}
Answer:"""

# Few-shot template (multiple examples + the question)
few_shot_template = """Question: {examples[0][0]}
Answer: {examples[0][1]}

Question: {examples[1][0]}
Answer: {examples[1][1]}

Question: {question}
Answer:"""

# TODO: Format the templates with your question and examples
zero_shot_prompt = zero_shot_template.format(question=question)
one_shot_prompt = one_shot_template.format(example_q=example_q, example_a=example_a, question=question)
# For few-shot, you'll need to format it with the examples list
few_shot_prompt = few_shot_template.format(examples=examples, question=question)

print("Zero-shot prompt:")
print(zero_shot_prompt)
print("\nOne-shot prompt:")
print(one_shot_prompt)
print("\nFew-shot prompt:")
print(few_shot_prompt)

Zero-shot prompt:
Question: What foods should be avoided by patients with gout?
Answer:

One-shot prompt:
Question: What are the symptoms of gout?
Answer: Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe.

Question: What foods should be avoided by patients with gout?
Answer:

Few-shot prompt:
Question: What are the symptoms of gout?
Answer: Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe.

Question: How is gout diagnosed?
Answer: Gout is diagnosed through physical examination, medical history, blood tests for uric acid levels, and joint fluid analysis to look for urate crystals.

Question: What foods should be avoided by patients with gout?
Answer:


## 3. Connecting to the LLM API

Next, implement a function to send prompts to an LLM API and get responses.

In [ ]:
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

with open("secrets.txt") as f:
    api_key = f.read().strip()
# HuggingFaceH4/zephyr-7b-beta
def get_llm_response(prompt, model_name="google/flan-t5-large", api_key=None):
    """Get a response from the LLM based on the prompt"""
    # TODO: Implement the get_llm_response function
    if api_key is None:
        with open("secrets.txt") as f:
            api_key = f.read().strip()

    if not api_key:
        raise ValueError("No Hugging Face API key provided.")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    # API_URL = f"https://api-inference.huggingface.co/models/{model_name}"
    # headers = {"Authorization": f"Bearer {api_key}"}
    
    payload = {
        "inputs": prompt,
        "parameters": {
            "max_new_tokens": 128,
            "do_sample": True,
            "return_full_text": False
        }
    }

    inputs = tokenizer(payload["inputs"], return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=256, do_sample=True)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

    
# TODO: Test your get_llm_response function with different prompts
response = get_llm_response(zero_shot_prompt)
print("Model prompt:")
print(zero_shot_prompt)
print("Model response:")
print(response)

response = get_llm_response(one_shot_prompt)
print("Model prompt:")
print(one_shot_prompt)
print("Model response:")
print(response)

response = get_llm_response(few_shot_prompt)
print("Model prompt:")
print(few_shot_prompt)
print("Model response:")
print(response)

Model prompt:
Question: What foods should be avoided by patients with gout?
Answer:
Model response:
foods rich in red meats that can cause a build up in your joints in itchy, painful skin
Model prompt:
Question: What are the symptoms of gout?
Answer: Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe.

Question: What foods should be avoided by patients with gout?
Answer:
Model response:
Glucose-rich foods like beans should be avoided by patients with gout.
Model prompt:
Question: What are the symptoms of gout?
Answer: Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe.

Question: How is gout diagnosed?
Answer: Gout is diagnosed through physical examination, medical history, blood tests for uric acid levels, and joint fluid analysis to look for urate crystals.

Question: What foods should be avoided by patients with gout?
Answer:
Model response:
Sugar, refined carbohydrate, and me

## 4. Comparing Prompting Strategies

Now, let's compare the different prompting strategies on a set of healthcare questions.

In [14]:
# List of healthcare questions to test
questions = [
    "What foods should be avoided by patients with gout?",
    "What medications are commonly prescribed for gout?",
    "How can gout flares be prevented?",
    "Is gout related to diet?",
    "Can gout be cured permanently?"
]


# TODO: Compare the different prompting strategies on these questions
# For each question:
# - Create prompts using each strategy
# - Get responses from the LLM
# - Store the results

results = {}

for question in questions:
    zero_shot_prompt = zero_shot_template.format(question=question)
    zero_shot_response = get_llm_response(zero_shot_prompt)

    one_shot_prompt = one_shot_template.format(example_q=example_q, example_a=example_a, question=question)
    one_shot_response = get_llm_response(one_shot_prompt)

    few_shot_prompt = few_shot_template.format(examples=examples, question=question)
    few_shot_response = get_llm_response(few_shot_prompt)

    results[question] = {
    "zero_shot": zero_shot_response,
    "one_shot": one_shot_response,
    "few_shot": few_shot_response
    }
    
print(results)

{'What foods should be avoided by patients with gout?': {'zero_shot': 'sulphites', 'one_shot': 'Grains like peas can also cause gout, but they should be avoided by diabetic patients, alcoholic patients, people with thyroid diseases, and other patients with gout.', 'few_shot': 'Gout causes painful bursts of fluid in joints and drained of blood, resulting in painful swelling on the capillaries of the affected joints. Patients with gout should avoid foods high in protein, such as red meat, cold cuts, pasta, and many foods high in fat.'}, 'What medications are commonly prescribed for gout?': {'zero_shot': 'antacids', 'one_shot': 'Asthaminas, colchicine, vinblastine, and tamiflu (Cialis) are commonly prescribed.', 'few_shot': 'Prescription drugs such as cyclophosphamide, doxazocine, or ketoconazole are routinely prescribed for gout.'}, 'How can gout flares be prevented?': {'zero_shot': 'If flares are rare, keep food low in uric acid.', 'one_shot': 'Keep your joints loose and dry', 'few_shot

## 5. Evaluating Responses

Create a simple evaluation function to score the responses based on the presence of expected keywords.

In [15]:
def score_response(response, keywords):
    """Score a response based on the presence of expected keywords"""
    # TODO: Implement the score_response function
    # Example implementation:
    response = response.lower()
    found_keywords = 0
    for keyword in keywords:
        if keyword.lower() in response:
            found_keywords += 1
    return found_keywords / len(keywords) if keywords else 0

# Expected keywords for each question
expected_keywords = {
    "What foods should be avoided by patients with gout?": 
        ["purine", "red meat", "seafood", "alcohol", "beer", "organ meats"],
    "What medications are commonly prescribed for gout?": 
        ["nsaids", "colchicine", "allopurinol", "febuxostat", "probenecid", "corticosteroids"],
    "How can gout flares be prevented?": 
        ["medication", "diet", "weight", "alcohol", "water", "exercise"],
    "Is gout related to diet?": 
        ["yes", "purine", "food", "alcohol", "seafood", "meat"],
    "Can gout be cured permanently?": 
        ["manage", "treatment", "lifestyle", "medication", "chronic"]
}

# TODO: Score the responses and calculate average scores for each strategy
# Determine which strategy performs best overall

for question, answers in results.items():
    keywords = expected_keywords.get(question, [])
    answers["zero_shot_score"] = score_response(answers["zero_shot"], keywords)
    answers["one_shot_score"] = score_response(answers["one_shot"], keywords)
    answers["few_shot_score"] = score_response(answers["few_shot"], keywords)

n = len(results)
avg_zero_shot = sum(entry["zero_shot_score"] for entry in results.values()) / n
avg_one_shot = sum(entry["one_shot_score"] for entry in results.values()) / n
avg_few_shot = sum(entry["few_shot_score"] for entry in results.values()) / n

print("Average scores by strategy:")
print(f"Zero-shot: {avg_zero_shot:.2f}")
print(f"One-shot: {avg_one_shot:.2f}")
print(f"Few-shot: {avg_few_shot:.2f}")

avg_scores = {
    "zero_shot": avg_zero_shot,
    "one_shot": avg_one_shot,
    "few_shot": avg_few_shot
}
best_strategy = max(avg_scores, key=avg_scores.get)
print(f"\nBest overall prompting strategy: {best_strategy} (average score: {avg_scores[best_strategy]:.2f})")

Average scores by strategy:
Zero-shot: 0.00
One-shot: 0.11
Few-shot: 0.14

Best overall prompting strategy: few_shot (average score: 0.14)


## 6. Saving Results

Save your results in a structured format for auto-grading.

In [16]:
# TODO: Save your results to results/part_3/prompting_results.txt
# The file should include:
# - Raw responses for each question and strategy
# - Scores for each question and strategy
# - Average scores for each strategy
# - The best performing strategy

# Example format:
"""
# Prompt Engineering Results

## Question: What foods should be avoided by patients with gout?

### Zero-shot response:
[response text]

### One-shot response:
[response text]

### Few-shot response:
[response text]

--------------------------------------------------

## Scores

```
question,zero_shot,one_shot,few_shot
what_foods_should,0.67,0.83,0.83
what_medications_are,0.50,0.67,0.83
how_can_gout,0.33,0.50,0.67
is_gout_related,0.80,0.80,1.00
can_gout_be,0.40,0.60,0.80

average,0.54,0.68,0.83
best_method,few_shot
```
"""
os.makedirs("results/part_3", exist_ok=True)
with open("results/part_3/prompting_results.txt", "w") as f:
    f.write("# Prompt Engineering Results\n\n")
    for question, answers in results.items():
        f.write(f"## Question: {question}\n\n")
        f.write("### Zero-shot response:\n")
        f.write(f"{answers['zero_shot']}\n\n")
        f.write("### One-shot response:\n")
        f.write(f"{answers['one_shot']}\n\n")
        f.write("### Few-shot response:\n")
        f.write(f"{answers['few_shot']}\n\n")
        f.write("-" * 50 + "\n\n")

    # Scores table
    f.write("## Scores\n\n")
    f.write("question,zero_shot,one_shot,few_shot\n")
    for question, answers in results.items():
        # Create a short question ID for the table
        qid = question.lower().replace("?", "").replace(" ", "_")[:20]
        zs = f"{answers['zero_shot_score']:.2f}"
        os = f"{answers['one_shot_score']:.2f}"
        fs = f"{answers['few_shot_score']:.2f}"
        f.write(f"{qid},{zs},{os},{fs}\n")
    f.write(f"\naverage,{avg_zero_shot:.2f},{avg_one_shot:.2f},{avg_few_shot:.2f}\n")
    f.write(f"best_method,{best_strategy}\n")
    f.write("```\n")


## Progress Checkpoints

1. **Prompting Templates**:
   - [ ] Create zero-shot template
   - [ ] Create one-shot template
   - [ ] Create few-shot template
   - [ ] Format templates with questions and examples

2. **LLM API Integration**:
   - [ ] Connect to the Hugging Face API
   - [ ] Test with different prompts
   - [ ] Handle API errors

3. **Comparison and Evaluation**:
   - [ ] Compare strategies on multiple questions
   - [ ] Score responses based on keywords
   - [ ] Determine the best strategy

4. **Results and Documentation**:
   - [ ] Save results in the required format
   - [ ] Document your findings

## What to Submit

1. Your implementation in a Python script `utils/prompt_comparison.py` that:
   - Defines the prompting templates
   - Connects to the Hugging Face API
   - Compares different prompting strategies
   - Scores and evaluates the responses

2. The results of your experiments in `results/part_3/prompting_results.txt` with the format shown above

The auto-grader will check:
1. That your results file contains the required sections
2. That your scoring logic correctly identifies keyword presence
3. That you've correctly calculated average scores
4. That you've identified the best performing method